In [1]:
### Spherical matching but with all stars in clusters
import argparse

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from astropy.table import Table
from astropy.io import fits
from astropy.coordinates import LSR, ICRS, SkyCoord
from astropy import units as u

from itertools import combinations, product

from scipy.spatial.distance import squareform

from tqdm import tqdm

kc19_fname = '/Users/aidanmcbride/Documents/Sagitta-Runaways/final6age.fits'
sagitta_fname = '/Users/aidanmcbride/Documents/Sagitta-Runaways/Sagitta_HDBSCAN_named_.fits'
output_fname = '/Users/aidanmcbride/Documents/Sagitta-Runaways/SphericalMotions/FullCluster/data_out/out-yymmdd.fits'

from FullCluster_functions import (maketable, clusterspatialparams, clustermotionparams, getalignment, getregion, get_dist,
                get_relative_velocity, get_tracebacktime, within_distance, bearing)

## Parameters 
yso_cut = 0.9 #select all YSO > .9
parallax_cut = 1 #select all parallax > 1 mas, or distance < 1000 pc

In [2]:
sagitta = maketable(sagitta_fname)
sagitta.rename(columns = {'pms': 'yso'})

for column in sagitta.columns:
    if sagitta[column].dtype == 'object':
        first = sagitta[column].iloc[0]
        first = str(first)
        if any(c.isalpha for c in first):
            sagitta[column] = sagitta[column].astype('str')
        elif '.' in first:
            sagitta[column] = sagitta['column'].astype(float)
        else:
            sagitta[column] = sagitta['column'].astype(int)



sagitta = sagitta.iloc[np.where((sagitta['yso'] > yso_cut) & (sagitta['parallax'] > parallax_cut))[0]]

#Define l1 column for interactions that may happen across the l = 0 discontinuity
if ('l1' in sagitta.columns):
    sagitta.drop(columns=['l1']) #reset this column if it already exists
sagitta['l1'] = sagitta['l']
over180 = np.where(sagitta['l1']>180)[0]
sagitta['l1'].iloc[over180] = sagitta['l1'].iloc[over180]-360

# Extract all labels from clustering, remove the unclustered (-1) sources
labels = np.sort(np.unique(sagitta['labels']))
if -1 in labels:
    labels = np.delete(labels, 0)
print('Labeled Clusters:', len(labels))


np.random.seed(1)
# downsample_by = 50
# sagitta = sagitta.iloc[np.random.choice(len(sagitta), int(len(sagitta)/downsample_by))]
# print(len(sagitta))

d1, d2 = (sagitta.iloc[np.where(sagitta['labels']!=-1)[0]], sagitta.iloc[np.where(sagitta['labels']==-1)[0]])

# alignments = np.loadtxt('/Users/aidanmcbride/Documents/Sagitta-Runaways/SphericalMotions/FullCluster/data_out/alignments_full.gz')
# # print(len(alignments))
# alignments = alignments.reshape((len(d1), len(d2))) #make to a matrix with rows and cols representing indices

# points1 = np.where(1 - np.abs(alignments) < 0.1)[0]
# points2 = np.where(1-np.abs(alignments) < 0.1)[1]
# np.savetxt('/Users/aidanmcbride/Documents/Sagitta-Runaways/SphericalMotions/FullCluster/data_out/ind1.gz', points1)
# np.savetxt('/Users/aidanmcbride/Documents/Sagitta-Runaways/SphericalMotions/FullCluster/data_out/ind2.gz', points2)
# print(alignments.shape)
# print(len(d1) * len(d2))

points1 = np.loadtxt('/Users/aidanmcbride/Documents/Sagitta-Runaways/SphericalMotions/FullCluster/data_out/ind1.txt')
points2 = np.loadtxt('/Users/aidanmcbride/Documents/Sagitta-Runaways/SphericalMotions/FullCluster/data_out/ind2.txt')#takes about 18min

# np.savetxt('/Users/aidanmcbride/Documents/Sagitta-Runaways/SphericalMotions/FullCluster/data_out/ind1.txt', points1)
# np.savetxt('/Users/aidanmcbride/Documents/Sagitta-Runaways/SphericalMotions/FullCluster/data_out/ind2.txt', points2)

Labeled Clusters: 609


In [2]:
# points1 = points1.astype(int)
points2 = points2.astype(int)
tab1 = d1.iloc[points1]
tab2 = d2.iloc[points2]

NameError: name 'points2' is not defined

In [1]:
print(len(tab1)/915559107)

NameError: name 'tab1' is not defined

In [ ]:
print(len(tab1))

SyntaxError: unexpected EOF while parsing (2977718154.py, line 1)

In [ ]:
def thing(row1, row2):
    l, b, vlsrl, vlsrb = (0,1,2,3)

    starl = row2[:,0]
    
    if 360 - starl < 90:
        l = 4
        row2[:,0] = starl - 360 #this might be a problem
    elif starl < 90:
        l = 4

    lon3 = (row1[:,l] - (row1[:,vlsrl] - row2[:,vlsrl])/1e3/3600)
    lat3 = (row1[:,b] - (row1[:,vlsrb] - row2[:,vlsrb])/1e3/3600)

    bearing_to = bearing(row1[:,b], row1[:,l], row2[:,b], row2[:,l])
    bearing_of = bearing(row1[:,b], row1[:,l], lat3, lon3)
    alignment = np.cos(bearing_of - bearing_to)

    return alignment

d1 = tab1[['l', 'b', 'vlsrl', 'vlsrb', 'l1']].values
d2 = tab2[['l', 'b', 'vlsrl', 'vlsrb', 'l1']].values #could use THESE to sort after doing pair_first, second
# alignments_array = np.zeros(len(d1) * len(d2))
# counter = 0
# for pair in tqdm(product(d1, d2), total = len(d1) * len(d2)):
    # a = thing(*pair)
    # alignments_array[counter] = a
    # counter += 1
    
alignments = thing(d1, d2)

NameError: name 'np' is not defined